In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
newapplication_df = application_df.drop(columns = ['EIN', 'NAME'])
newapplication_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [7]:
# Determine the number of unique values in each column.
uniquevaluecounts = newapplication_df.nunique()
uniquevaluecounts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
valuecnts = newapplication_df['APPLICATION_TYPE'].value_counts()
valuecnts_df = pd.DataFrame(valuecnts)
valuecnts_df.dtypes
to_replace = []
valcnts_not_reqd= newapplication_df['APPLICATION_TYPE'].value_counts() < 500
valcnts_not_reqd[8:]
valuecnts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ['T9','T12', 'T13', 'T14', 'T25', 'T29', 'T17','T2', 'T15']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
new_classificationcnts = newapplication_df['CLASSIFICATION'].value_counts()
classificationcnts = pd.DataFrame(data=new_classificationcnts)
classificationcnts.reset_index()
newclassificationcnts = classificationcnts.loc[classificationcnts['CLASSIFICATION']>1, :]
newclassificationcnts

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
other_classifications=classificationcnts.loc[classificationcnts['CLASSIFICATION']<1000, 'CLASSIFICATION']
OC_df = pd.DataFrame(other_classifications)
classifications_to_replace.append(OC_df.index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
categoryto_numeric_df = pd.get_dummies(newapplication_df)
categoryto_numeric_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
#filter dataframe for only the active status applications and use is_successful for y
active_status=categoryto_numeric_df.loc[categoryto_numeric_df['STATUS']==1, :]
active_status.head()
y=active_status['IS_SUCCESSFUL']
X=active_status.drop(columns='IS_SUCCESSFUL')
X

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify =y)
X_train.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
2686,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
15084,1,5000,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
25243,1,250095,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
20659,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
32087,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
print(y_train)
print(y_test)

2686     1
15084    1
25243    1
20659    0
32087    1
        ..
32479    0
32505    0
9204     1
14407    1
24717    1
Name: IS_SUCCESSFUL, Length: 25720, dtype: int64
15585    1
13092    0
22724    0
30764    1
9230     1
        ..
26877    1
33382    1
14851    1
32871    1
28845    1
Name: IS_SUCCESSFUL, Length: 8574, dtype: int64


In [16]:
X_test

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
15585,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13092,1,49811,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
22724,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30764,1,7527463,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
9230,1,57256542,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26877,1,158368,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
33382,1,44618,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
14851,1,6078,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
32871,1,2919956,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 0.        , -0.03148233, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588],
       [ 0.        , -0.03102552, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588],
       [ 0.        , -0.03148233, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588],
       ...,
       [ 0.        , -0.03147134, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588],
       [ 0.        , -0.00176684, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588],
       [ 0.        , -0.03148233, -0.12584875, ..., -0.07450937,
         0.02858588, -0.02858588]])

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="ELU", input_dim=116))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                9360      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 18301 (71.49 KB)
Trainable params: 18301 (71.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
train_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=10)
train_model

Epoch 1/100
2572/2572 [==============================] - 9s 3ms/step - loss: 0.5654 - accuracy: 0.7227
Epoch 2/100
2572/2572 [==============================] - 6s 2ms/step - loss: 0.5532 - accuracy: 0.7290
Epoch 3/100
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5501 - accuracy: 0.7311
Epoch 4/100
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5481 - accuracy: 0.7309
Epoch 5/100
2572/2572 [==============================] - 6s 2ms/step - loss: 0.5472 - accuracy: 0.7331
Epoch 6/100
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5464 - accuracy: 0.7329
Epoch 7/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5455 - accuracy: 0.7338
Epoch 8/100
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5444 - accuracy: 0.7343
Epoch 9/100
2572/2572 [==============================] - 6s 2ms/step - loss: 0.5440 - accuracy: 0.7358
Epoch 10/100
2572/2572 [==============================] - 6s 2ms/step - l

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5616 - accuracy: 0.7229 - 512ms/epoch - 2ms/step
Loss: 0.5615824460983276, Accuracy: 0.7228831052780151


In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
tn = tf.keras.models.Sequential()
# First hidden layer
tn.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=116))
# Second hidden layer
tn.add(tf.keras.layers.Dense(units=10, activation="sigmoid", input_dim = 40))
# Third hidden layer
tn.add(tf.keras.layers.Dense(units=10, activation="sigmoid", input_dim= 50))
# Fourth hidden layer
tn.add(tf.keras.layers.Dense(units=10, activation="sigmoid", input_dim = 30))
# Output layer
tn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
tn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                1170      
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1511 (5.90 KB)
Trainable params: 1511 (5.90 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model
tn.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
train_model = tn.fit(X_train_scaled, y_train, epochs=100, batch_size=10)
train_model

Epoch 1/100
2572/2572 [==============================] - 8s 3ms/step - loss: 0.5986 - accuracy: 0.7024
Epoch 2/100
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5712 - accuracy: 0.7322
Epoch 3/100
2572/2572 [==============================] - 8s 3ms/step - loss: 0.5688 - accuracy: 0.7314
Epoch 4/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5655 - accuracy: 0.7344
Epoch 5/100
2572/2572 [==============================] - 6s 2ms/step - loss: 0.5573 - accuracy: 0.7320
Epoch 6/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5530 - accuracy: 0.7343
Epoch 7/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5502 - accuracy: 0.7337
Epoch 8/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5487 - accuracy: 0.7348
Epoch 9/100
2572/2572 [==============================] - 5s 2ms/step - loss: 0.5473 - accuracy: 0.7352
Epoch 10/100
2572/2572 [==============================] - 5s 2ms/step - l

In [25]:
model_loss, model_accuracy = tn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7296 - 471ms/epoch - 2ms/step
Loss: 0.5550773739814758, Accuracy: 0.7296477556228638
